In [143]:
# -*- coding: utf-8 -*-
# -*- author : Vincent Roduit -*-
# -*- date : 2023-11-25 -*-
# -*- Last revision: 2023-11-25 -*-
# -*- python version : 3.11.6 -*-
# -*- Description: Notebook that summarize results-*-

# <center> CS -433 Machine Learning </center>
## <center> Ecole Polytechnique Fédérale de Lausanne </center>
### <center>Road Segmentation </center>
--- 

### Preparing environment for Google Colaboratory

### Imports

In [144]:
#import libraries
%matplotlib inline
import sklearn as sk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#import files
from load_datas import*
from constants import*
from visualization import*
from data_processing import*
from data_augmentation import*
from preprocessing_helper import *
from post_processing import*
from training_utils import*

# pytorch
import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Load and preprocess datas

In [145]:
imgs, gt_imgs = load_datas(NB_IMAGES_MAX)

imgs_train = imgs[:80]
gt_imgs_train = gt_imgs[:80]
imgs_test = imgs[80:100]
gt_imgs_test = gt_imgs[80:100]

In [146]:
from new_pre_helper import*

In [147]:
WINDOW = 60

In [151]:
rotated_imgs, rotated_gt_imgs = rotate_imgs_train(imgs, gt_imgs)

Rotation for 45 degrees
Rotation for 135 degrees
Rotation for 225 degrees
Rotation for 315 degrees


In [157]:
X, y = image_generator(imgs_train, gt_imgs_train, window_size=WINDOW, nb_batches=200, batch_size=32, upsample=True)
# x_val, y_val = image_generator(imgs_test, gt_imgs_test, window_size=WINDOW_SIZE, nb_batches=50, batch_size=32, upsample=True)
x_val, y_val = create_windows_gt(imgs_test, gt_imgs_test, augm_patch_size=WINDOW_SIZE)

In [179]:
X, y = new_image_generator(
    imgs_train, 
    gt_imgs_train,
    rotated_imgs,
    rotated_gt_imgs,
    window_size=WINDOW, 
    num_samples=128000, 
    batch_size=32)

x_val, y_val = new_create_windows_gt(
    imgs_test, 
    gt_imgs_test, 
    augm_patch_size=WINDOW)

Batch 1/4000
Batch 401/4000
Batch 801/4000
Batch 1201/4000
Batch 1601/4000
Batch 2001/4000
Batch 2401/4000
Batch 2801/4000
Batch 3201/4000
Batch 3601/4000
end process...


In [169]:
train_dataloader = DataLoader(
    dataset=list(zip(X, y)),
    batch_size=32,
    shuffle=True,
    num_workers=0)

validate_dataloader = DataLoader(
    dataset=list(zip(x_val, y_val)),
    batch_size=32,
    shuffle=False,
    num_workers=0)

In [ ]:
#Epoch 1/20, Loss: 0.6045116235923768, Validation Accuracy: 0.6547, F1 score: 0.5090

## After the first change in image_generator (only rotation)
#Epoch 1/20, Loss: 0.5709283715629577, Validation Accuracy: 0.7626, F1 score: 0.4797
## After second change in image_generator (rotation + flip)
#Epoch 1/20, Loss: 0.6519827550888061, Validation Accuracy: 0.6142, F1 score: 0.5411

In [170]:
# Set random seed for reproducibility
torch.manual_seed(0)

from cnn import *

cnn = Basic_CNN(WINDOW)

# Define the loss function, optimizer and scheduler
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
criterion = nn.BCEWithLogitsLoss()

# Train the model
cnn.train_model(optimizer, scheduler, criterion, train_dataloader, validate_dataloader, num_epochs=20)

Epoch 1/20, Loss: 0.6116752460670472, Validation Accuracy: 0.7010, F1 score: 0.5630


KeyboardInterrupt: 